In [216]:
#import netgen.gui
from ngsolve import *
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import inv

In [2]:
from netgen.geom2d import unit_square
mesh = Mesh(unit_square.GenerateMesh(maxh=0.05))
Draw (mesh)

In [3]:
fes = H1(mesh, order=3,dirichlet=".*")
gfu_a = GridFunction(fes,name = "u_a")
gfu_b = GridFunction(fes,name = "u_b")
print(len(gfu_a.vec))

4342


In [ ]:
r = u.vec.CreateVector()
w = u.vec.CreateVector()
Mu = u.vec.CreateVector()
Au = u.vec.CreateVector()
Mw = u.vec.CreateVector()
Aw = u.vec.CreateVector()

for i in range(20):
    Au.data = a.mat * u.vec
    Mu.data = m.mat * u.vec
    auu = InnerProduct(Au, u.vec)
    muu = InnerProduct(Mu, u.vec)
    # Rayleigh quotient
    lam = auu/muu
    print (lam / (math.pi**2))
    # residual
    r.data = Au - lam * Mu
    w.data = pre.mat * r.data
    w.data = 1/Norm(w) * w
    Aw.data = a.mat * w
    Mw.data = m.mat * w

    # setup and solve 2x2 small eigenvalue problem
    asmall = Matrix(2,2)
    asmall[0,0] = auu
    asmall[0,1] = asmall[1,0] = InnerProduct(Au, w)
    asmall[1,1] = InnerProduct(Aw, w)
    msmall = Matrix(2,2)
    msmall[0,0] = muu
    msmall[0,1] = msmall[1,0] = InnerProduct(Mu, w)
    msmall[1,1] = InnerProduct(Mw, w)
    # print ("asmall =", asmall, ", msmall = ", msmall)


    eval,evec = scipy.linalg.eigh(a=asmall, b=msmall)
    # print (eval, evec)
    u.vec.data = float(evec[0,0]) * u.vec + float(evec[1,0]) * w

Draw (u)

In [349]:
def SolveCG(A,C, u_grid,f_grid,maxsteps, tol, printrate):
    d = u_grid.CreateVector()
    p = u_grid.CreateVector()
    alpha = u_grid.CreateVector()
    beta = u_grid.CreateVector()
    temp1 = u_grid.CreateVector()
    temp2 = u_grid.CreateVector()
    
    d.data = f_grid - A*u_grid
    d.data = Projector(fes.FreeDofs(), True)*d
    #p.data = C * d.data
    
    temp1 = d.CreateTranspose()*p.data
    #temp2 = A*d.data
    #temp2 = np.transpose(d)*temp2
    
    #alpha = temp1.data/temp2.data
    #print(temp2)
    #temp2.data = A*p
    #temp2.data = np.transpose(p).data*temp
    
    #print(temp.data)
    #
    #temp.data = np.transpose(p)*temp.data
    #alpha = (np.transpose(d)*p.data)/(np.transpose(d)*A*p.data)
    #print(np.transpose(d)*A.vec)
    #temp = p.data.CreateTranspose()
    #print(p.data.CreateTranspose())
    #Mw = u.vec.CreateVector()
    #Aw = u.vec.CreateVector()
    
    
    #print(u_grid)
    #ngs_temp = A.CreateColVector()
    #temp = A.CreateColVector()
    #Cinv_temp = C.Inverse()
    #ngs_temp.data = f_grid - A*u_grid 
                
    #d0 = A.CreateColVector()
    #p0 = C.CreateColVector()
    #d0.data = Projector(fes.FreeDofs(), True)*ngs_temp
    
    
    #p0.data = C * d0.data
    #temp = d0.data.CreateTranspose() * p0.data
    
    #print(p0[1])
    #p0.data = Cinv_temp * d0.data
    #C.Inverse()
    #print(d0.data)
    #u = np.zeros(len(u_grid.vec))
    #f = np.zeros(len(f_grid.vec))
    #rows,cols,vals = A.COO()
    #A = sp.csr_matrix((vals,(rows,cols)))
    #rows,cols,vals = C.COO()
    #C = sp.csr_matrix((vals,(rows,cols)))
    
    #f = f_grid.vec
    #print(C.
    #rows,cols,vals = f.COO()
    #print(type(f.vec))
    #print(A.dot(f))
    #vu = u_grid.vec
    #res = vu.CreateVector()
    #res.vec = A.mat.Inverse(fes.FreeDofs()) * f.vec
    #print(res.vec)
    
    
    #uN[:,i] = np.linalg.solve(KN-omegaN[i]**2*MN, fN)
    #new = uN[:,i].dot(np.transpose(V))
    #gif = GridFunction(fes,name = "gif")
    #print(new)
    #gif.vec.FV().NumPy()[:] = new
    
    
    #vu = u_grid.vec
    #u_k = u_grid.vec.CreateVector()
    
    #np.zeros(len(u_grid.vec))
                
    #ngs_temp = self.k_blf.mat.CreateColVector()
    #ngs_temp.data = k_blf.mat*ured.vec - _omega*_omega*m_blf.mat*ured.vec - f.vec
                
    #ngs_temp2 = self.k_blf.mat.CreateColVector()
    #ngs_temp2.data = Projector(self.fes.FreeDofs(), True)*ngs_temp
                
    
    #rows,cols,vals = A.COO()
    #A = sp.csr_matrix((vals,(rows,cols)))
    #A = sp.csr_matrix.toarray(A)
    
#    rows,cols,vals = C.COO()
#    C = sp.csr_matrix((vals,(rows,cols)))
#    C = sp.csr_matrix.toarray(C)
    #print(C)

In [350]:
ua = fes.TrialFunction()
va = fes.TestFunction()

Aa = BilinearForm(fes)
Fa = LinearForm(fes)

Aa += ua*va*dx

Fa += va*dx

#u = vu.CreateVector()


Aa.Assemble()
Fa.Assemble()

#rows,cols,vals = Aa.mat.COO()
#Aa = sp.csr_matrix((vals,(rows,cols)))
#Aa = sp.csr_matrix.toarray(Aa)

#print(Aa)
c = Preconditioner(Aa,"local",test = True)
#Preconditioner(
c.Update()

#C = CreateMatrix(c)
#print(c.data)
SolveCG(Aa.mat,c.mat,gfu_a.vec,Fa.vec,10,int(1e-8),True)

#c.Update()

#gfu_a.vec.data = Aa.mat.Inverse(fes.FreeDofs()) * Fa.vec

#Draw(gfu_a)

AttributeError: 'ngsolve.la.BaseVector' object has no attribute 'CreateTranspose'

In [325]:
#CG Method
    errorCG = []
    NormresidualsCG = []
    x0 = np.zeros(n_array[i]*n_array[i])
    r_l = b-np.dot(A,x0)
    d_l = r_l
    tol = 1e-16
    for k in range(l_max):
        
        alpha = np.dot(r_l,r_l)/np.dot(d_l,np.dot(A,d_l))
        r_l1 = r_l - alpha*np.dot(A,d_l)
        x0 = x0 + alpha*d_l
        beta = -np.dot(r_l1,r_l1)/np.dot(r_l,r_l)
        d_l = r_l1 - beta*d_l
        ERROR = np.sqrt(np.dot(x0-xstar,np.dot(A,x0-xstar)))
        NormresidualsCG.append(np.sqrt(np.dot(b-np.dot(A,x0),b-np.dot(A,x0))))
        errorCG.append(ERROR)
        
        if ERROR < tol:
            break
        
        
        r_l = r_l1

IndentationError: unexpected indent (<ipython-input-325-1d4ed9c26738>, line 2)